In [1]:
%load_ext nb_black

from pipeline import Pipeline
import json
from pipeline import build_csv
import io
import datetime

<IPython.core.display.Javascript object>

In [2]:
pipeline = Pipeline()

<IPython.core.display.Javascript object>

In [3]:
@pipeline.task()
def file_to_json():
    with open("hn_stories_2014.json") as f:
        stories = json.load(f)["stories"]
    return stories

<IPython.core.display.Javascript object>

In [4]:
@pipeline.task(depends_on=file_to_json)
def filter_stories(stories):
    return (
        story
        for story in stories
        if (story["points"] > 50)
        and (story["comments"] > 1)
        and not (story["title"].startswith("Ask HN"))
    )

<IPython.core.display.Javascript object>

In [7]:
@pipeline.task(depends_on=filter_stories)
def json_to_csv(stories):
    stories = [
        [
            story["objectID"],
            datetime.datetime.fromiso(story["created_at"]),
            story["url"],
            story["points"],
            story["title"],
        ]
        for story in stories
    ]
    return build_csv(
        stories,
        header=["objectID", "created_at", "url", "points", "title"],
        file=io.StringIO(),
    )

<IPython.core.display.Javascript object>